In [19]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [20]:
data = pd.read_csv('data_good_sentiment_final.csv',names=['text',"label"])

In [21]:
x_train,x_test,y_train, y_test= train_test_split(data['text'],data['label'],test_size=0.2, random_state=42)

In [22]:
tokenizer = Tokenizer(num_words=5000,lower=True)
tokenizer.fit_on_texts(x_train)
tokenizer.fit_on_texts(x_test)
word_index = tokenizer.word_index

In [23]:
X_train_tokenizer = pad_sequences(tokenizer.texts_to_sequences(x_train),maxlen=250)
X_test_tokenizer =  pad_sequences(tokenizer.texts_to_sequences(x_test),maxlen=250)

In [24]:
enc = OneHotEncoder()
Y_train_Labels = enc.fit_transform(y_train.values.reshape(-1, 1)).toarray()
Y_test_Labels = enc.transform(y_test.values.reshape(-1, 1)).toarray()

In [25]:
import mxnet as mx
from mxnet import gluon, autograd,nd
from mxnet.gluon import nn

In [26]:
type(X_train_tokenizer)

numpy.ndarray

In [27]:
type(Y_train_Labels)

numpy.ndarray

In [28]:
x_train=mx.nd.array(X_train_tokenizer)
x_test=mx.nd.array(X_test_tokenizer)
y_train=mx.nd.array(Y_train_Labels)
y_test=mx.nd.array(Y_test_Labels)

In [29]:
# from keras.utils import to_categorical
# y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)

In [30]:

model = mx.gluon.nn.Sequential()

with model.name_scope():    
    model.embed = mx.gluon.nn.Embedding(5000, 100)
    model.add(mx.gluon.rnn.LSTM(100, layout = 'NTC',bidirectional=True,dropout=0.2))
    model.add(mx.gluon.nn.Dense(100,activation='relu'))
    model.add(mx.gluon.nn.Dense(4))

In [31]:
model.summary

<bound method Block.summary of Sequential(
  (embed): Embedding(5000 -> 100, float32)
  (1): LSTM(None -> 100, NTC, dropout=0.2, bidirectional)
  (2): Dense(None -> 100, Activation(relu))
  (3): Dense(None -> 4, linear)
)>

In [32]:
ctx=mx.cpu(0)

In [33]:
model.initialize(mx.init.Xavier(),ctx)

In [34]:
trainer = gluon.Trainer(
    params= model.collect_params(),
    optimizer= 'adam',
    optimizer_params={'learning_rate':1}
)

In [35]:
metric = mx.metric.Accuracy()
sigmoid_binary_cross_entropy_loss = gluon.loss.SigmoidBinaryCrossEntropyLoss()

In [36]:
num_epochs = 3
for epoch in range(num_epochs):
    for x,y in zip(x_train,y_train):
        x=x.as_in_context(ctx)
        # print(type(x))
        # print(x.shape)
        y=y.as_in_context(ctx)
        # print(type(y))
        # print(y.shape)
        # print(y)
        with autograd.record():
            y_hat= model(x.reshape(1,-1))
            # print(y_hat[0].shape)
            # print(y_hat[0])
            loss=sigmoid_binary_cross_entropy_loss(y_hat[0],y)
        loss.backward()
        metric.update(y_hat[0],y)
        trainer.step(batch_size=128)
    name,acc=metric.get()
    print(acc)
    metric.reset()

0.26195148290575776
0.26220170679028604
0.26220170679028604
